# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [1]:
#!pip install xlrd
#!pip install openpyxl
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

def generate_energy_df():
    """ 
    Loads and cleans the Energy Indicators dataset.

    Return:
        energy_df (DataFrame): A cleaned DataFrame with columns 
        ['Country','Energy Supply', 'Energy Supply per Capita', 
        '% Renewable']
    """
    # YOUR CODE HERE load dataframe and remove header and footer
    energy_df = pd.read_excel(
        './assets/Energy Indicators.xls',
        skiprows=17,
        skipfooter=38,
        usecols="C:F",
        names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    )

    # YOUR CODE HERE transform data
    energy_df['Energy Supply'] *= 1_000_000
    energy_df = energy_df.replace({'...': np.nan})

    # YOUR CODE HERE transform Country column

    energy_df['Country'] = energy_df['Country'].str.replace(r'\s*\(.*\)\s*', '', regex=True)
    energy_df['Country'] = energy_df['Country'].str.replace(r'\d+', '', regex=True)

    energy_df['Country'] = energy_df['Country'].replace({
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    })

    return energy_df

def get_GDP_df():
    """ 
    Loads and cleans the World Bank GDP dataset.

    Return: 
        A cleaned DataFrame with columns ['Country', 
        '2006'-'2015']
    """
    gdp_df = pd.read_csv('./assets/world_bank.csv', skiprows=4)
    gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)



    # YOUR CODE HERE load data, remove header, adjust column names
    country_renames = {
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong",
    }
    gdp_df['Country'].replace(country_renames, inplace=True)


    # YOUR CODE HERE transform Country column 
    gdp_df = gdp_df.drop(gdp_df.columns[1:-10], axis=1)
    return gdp_df

def get_scimago_df():
    """ 
    This function loads the Scimago Journal and Country Rank dataset 
    related to energy engineering and power technology. 
    
    Return:
        scimago_df(DataFrame): Returns the entire DataFrame (scimago_df)
        as it only contains 15 rows (top 15 countries by rank).

    """
    # YOUR CODE HERE load data
    scimago_df = pd.read_excel('assets/scimagojr-3.xlsx')
    return scimago_df

def answer_one():
    """ 
    This function merges Energy, GDP, and Scimago Journal Rank datasets
    into one DataFrame.
    
    Returns:
        df (DataFrame): A merged DataFrame with columns ['Rank', 'Documents',
        'Citable documents', 'Citations', 'Self-citations', 
        'Citations per document', 'H index', 'Energy Supply', 
        'Energy Supply per Capita', '% Renewable', '2006'-'2015']
    """
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    df = pd.merge(energy_df, gdp_df, on='Country', how='inner')
    df = pd.merge(scimago_df.iloc[:15], df, on='Country', how='inner')
    df.set_index('Country', inplace=True)
    df.sort_values('Rank', inplace=True)
    return df

In [2]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [84]:
# space for proffesor tests


## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [85]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [3]:
def answer_two():
    """ 
    This function calculates the number of entries lost after 
    merging datasets but before reducing to the top entries.

    Return:
        lost_entries (int): A number of entries lost in the 
        merging process before reducing to top items
    """
    merged_df = answer_one()
    complete = pd.merge(generate_energy_df(), get_GDP_df(), on='Country', how='outer')
    complete = pd.merge(get_scimago_df(), complete, on='Country', how='outer')

    return len(complete) - len(merged_df)

In [4]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [88]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [5]:
def answer_three():
    """ 
    This function calculates the average GDP over the 
    last 10 years for each country and retrieves the 
    top 15 countries.
    
    Returns:
        avg_gdp (Series): A Series with 15 countries and 
        their average GDP values sorted in descending order.

    """
    avg_gdp = answer_one().iloc[:, 10:].mean(axis=1).sort_values(ascending=False)
    return avg_gdp
answer_three().head(50)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [6]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [91]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [7]:
def answer_four():
    """
    This function calculates the GDP change over the 
    10-year span for the country ranked 6th in average
    GDP.
    
    Returns:
        gdp_change (float): GDP change over the 10-year 
        span for the 6th ranked country.
    """
    avg_gdp = answer_three()
    sixth_country = avg_gdp.index[5]  
    gdp_df = get_GDP_df().set_index('Country')
    gdp_change = gdp_df.loc[sixth_country].iloc[-1] - gdp_df.loc[sixth_country].iloc[0]
    gdp_change = float(gdp_change)
    return gdp_change

print(answer_four())

246702696075.3999


In [8]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [ ]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [9]:
def answer_five():
    """ 
    This function calculates the mean energy supply 
    per capita across all countries.

    Returns:
        mean_energy_supply_per_capita (float): Mean 
        energy supply per capita.
    """
    energy_df = generate_energy_df()
    mean_energy_supply_per_capita = float(energy_df['Energy Supply per Capita'].mean())
    return mean_energy_supply_per_capita
print(answer_five())

90.66666666666667


In [10]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [ ]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [11]:
def answer_six():
    """ 
    This function finds the country with the maximum
    percentage of renewable energy and returns both 
    the country name and the percentage.
    
    Returns:
        tuple(max_renewable_country (str), 
        max_renewable_percentage (float)): A tuple with 
        the name of the country that has the maximun 
        % Renewable and the percentage of this country.
    """
    maximun_renewable = answer_one()['% Renewable'].idxmax()
    percentage = float(answer_one()['% Renewable'].max())
    return (maximun_renewable, percentage)

print(answer_six())


('Brazil', 69.64803)


In [12]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [13]:
def answer_seven():
    """ 
    This function computes a new column in the dataset 
    obtained from answer_one() that represents the ratio 
    of Self-Citations to Total Citations for each country
    
    Returns:
        tuple: (country_name (str), citation_ratio (float)):
        A tuple containing the name of the country with the 
        highest Citation Ratio and the corresponding ratio 
        value.
    """
    df = answer_one()
    df["Citation Ratio"] = df["Self-citations"]/df["Citations"]
    maximun_citation_ratio = df[df["Citation Ratio"] == df["Citation Ratio"].max()]
    answer = maximun_citation_ratio["Citation Ratio"].index[0], maximun_citation_ratio["Citation Ratio"].values[0]

    return answer
print(answer_seven())

('China', 0.6893126179389422)


In [14]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [15]:
def answer_eight():
    """ 
    This function calculates the estimated population using
    Energy Supply and Energy Supply per capita, and 
    identifies the third most populous country according to 
    this estimate.

    Return:
        third_most_populous_country(str): the name of the 
        country ranking third in terms of estimated population.
    """
    df = answer_one()
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df.sort_values(by='Population', ascending=False, inplace=True)
    third_most_populous_country = df.index[2]
    return third_most_populous_country
print(answer_eight())


United States


In [16]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [17]:
def answer_nine():
    """ 
    This function calculates the correlation between 
    citable documents per capita and energy supply per 
    capita using Pandas' .corr() method, which employs 
    Pearson's correlation.

    Return: 
        correlation(float):A single number representing 
        Pearson's correlation coefficient between citable 
        documents per capita and energy supply per capita.
    """
    df = answer_one()
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['Citable documents per capita'] = df['Citable documents'] / df['Population']
    correlation = df['Citable documents per capita'].corr(df['Energy Supply per Capita'])
    
    return correlation
print(answer_nine())

0.7940010435442946


In [18]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [ ]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [19]:
def answer_ten():
    """ 
    This function creates a new column indicating whether
    a country's '% Renewable' value is above or equal to 
    the median of all countries in the top 15, and returns
    a series named 'high_renew' indexed by the country 
    name, sorted in ascending order of rank.

    Return: 
        high_renew(Series): A Series where each country in 
        the top 15 is assigned a binary value indicating 
        whether its '% Renewable' is above the median.
    """
    df = answer_one()
    median_renewable = df['% Renewable'].median()
    df['high_renew'] = df['% Renewable'].apply(lambda x: 1 if x >= median_renewable else 0)
    high_renew = df['high_renew']
    return high_renew
print(answer_ten())

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: high_renew, dtype: int64


In [20]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [ ]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [21]:
def answer_eleven():
    """ 
    This function groups countries by continent using the
    ContinentDict dictionary, and creates a DataFrame 
    displaying sample size (number of countries), sum, mean,
    and standard deviation of the estimated population for 
    each continent.

    Return:
        continent_stats(DataFrame): A DataFrame where each row
        represents a continent, and columns are sample size, 
        sum, mean, and standard deviation of the estimated 
        population.

    """
    df = answer_one()
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']

    #Dictionary with continent
    ContinentDict = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }

    # Assign continents based on ContinentDict
    df['Continent'] = df.index.map(lambda x: ContinentDict[x])

    # Group by continent and calculate statistics
    continent_stats = df.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])

    # Rename index to 'Continent'
    continent_stats.index.name = 'Continent'
    
    return continent_stats

answer_eleven().head()

,size,sum,mean,std
Continent,,,,
Asia,5,2898666386.6106,579733277.32212,6.790979e+08
Australia,1,23316017.316017,23316017.316017,NaN
Europe,6,457929667.216372,76321611.202729,3.464767e+07
North America,2,352855249.48025,176427624.740125,1.996696e+08
South America,1,205915254.237288,205915254.237288,NaN


In [22]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [ ]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [23]:
def answer_twelve():
    """ 
    This function bins the '% Renewable' column using pd.cut()
    and groups the Top 15 countries by continent and these new 
    '% Renewable' bins. It returns a series with a MultiIndex 
    of continent and '% Renewable' bins, indicating the number 
    of countries in each group.

    Return:
        df(Series): A Series with a MultiIndex where the first 
        level is the continent and the second level are the 
        '% Renewable' bins, indicating the count of countries 
        in each group.
    """
    ContinentDict  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }

    df = answer_one()
    df = df.reset_index()
    df["Continent"] = df["Country"].map(ContinentDict)

    df["% Renewable"] = pd.cut(df["% Renewable"], 5)
    df = df.groupby(["Continent", "% Renewable"]).agg({"Country": pd.Series.nunique})
    df = df["Country"].dropna()
  
    return df
answer_twelve().head(15)

Continent      % Renewable     
Asia           (2.212, 15.753]     4.0
               (15.753, 29.227]    1.0
Australia      (2.212, 15.753]     1.0
Europe         (2.212, 15.753]     1.0
               (15.753, 29.227]    3.0
               (29.227, 42.701]    2.0
North America  (2.212, 15.753]     1.0
               (56.174, 69.648]    1.0
South America  (56.174, 69.648]    1.0
Name: Country, dtype: float64

In [24]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

In [ ]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [25]:
def answer_thirteen():
    """ 
    This function converts the Population Estimate series to a
    string with thousands separators (using dots), preserving 
    all significant digits without rounding.

    Return:
        pop_est(Series): A Series where each index is a country 
        name and each value is the population estimate formatted 
        as a string.
    """
    df = answer_one()
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    
    # Format Population column with thousands separator
    pd.options.display.float_format = '{:,.2f}'.format
    pop_est = df['Population'].apply(lambda x: '{:,.2f}'.format(x))
    
    return pop_est
print(answer_thirteen())

Country
China                 1,367,645,161.29
United States           317,615,384.62
Japan                   127,409,395.97
United Kingdom           63,870,967.74
Russian Federation      143,500,000.00
Canada                   35,239,864.86
Germany                  80,369,696.97
India                 1,276,730,769.23
France                   63,837,349.40
South Korea              49,805,429.86
Italy                    59,908,256.88
Spain                    46,443,396.23
Iran                     77,075,630.25
Australia                23,316,017.32
Brazil                  205,915,254.24
Name: Population, dtype: object


In [26]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [ ]:
# space for proffesor tests